# 第6章：仮説検証型データ分析

## 6.1 仮説検証のための事前準備

### 6.1.3 ショッパー属性リストの作成

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

df1 = pd.read_csv("../input/gi_train_mm10.csv") # 10月のデータ
df2 = pd.read_csv("../input/gi_train_mm11.csv") # 11月のデータ

df_tmp = pd.concat([df1, df2])
df = df_tmp[df_tmp["customer_id"].notna()].copy()

df_et1 = df.loc[(df["event_type"] == 1)]
df_et2 = df.loc[(df["event_type"] == 2)]
df_et3 = df.loc[(df["event_type"] == 3)]

shopper_attribute_list = df[["customer_id","gender","age","age_category"]].drop_duplicates().set_index("customer_id")
shopper_attribute_list

### 6.1.4 ショッパー接触回数リストの作成

In [ ]:
shopper_contact_list = df.groupby("customer_id")[["num_touch"]].sum().astype(int).rename(columns={"num_touch" : "接触回数"})
shopper_contact_list

### 6.1.5 ショッパー購入回数リストの作成

In [ ]:
shopper_buy_list = df.groupby("customer_id")[["buy_flag"]].sum().astype(int).rename(columns={"buy_flag" : "購入回数"})
shopper_buy_list

### 6.1.6 ショッパー滞在時間リストの作成

In [ ]:
shopper_time_duration_list = df_et1[["customer_id","time_duration"]].set_index("customer_id")
shopper_time_duration_list

### 6.1.7 ショッパー情報一覧の作成

In [ ]:
df_shopper = pd.concat([shopper_attribute_list, shopper_contact_list, shopper_buy_list, shopper_time_duration_list], axis=1)
df_shopper

## 6.2 仮説1「女性の方がいろいろと商品を検討してそう」の検証

### 6.2.2 データで検証

In [ ]:
df_shopper.groupby("gender")["接触回数"].mean()

## 6.3 仮説2「女性の方が長い時間比較検討してそう」の検証

### 6.3.2 データで検証

In [ ]:
df_shopper.groupby("gender")["time_duration"].describe()

### 6.3.5 仮説検定の進め方②　有意水準を設定しp値を算出する

In [ ]:
import scipy.stats as st
xa = df_shopper[df_shopper["gender"] == "man"]["time_duration"]
xb = df_shopper[df_shopper["gender"] == "woman"]["time_duration"]
t, p = st.ttest_ind(xa, xb, equal_var=False)
print("t値=", t)
print("p値=", p)

## 6.4 仮説3「若い人はあまり商品を検討していない」の検証

### 6.4.2 データで検証

In [ ]:
df_shopper.groupby("age")["接触回数"].describe()

## 6.5 仮説4「『雪のしずく』は『岩清水』と比較検討されている」の検証

### 6.5.2 データで検証

In [ ]:
#「雪のしずく550ml」に接触したショッパーの人物IDのリストを作成
list_tdr = df_et3[df_et3["product_name"] == "雪のしずく550ml"]["customer_id"].drop_duplicates()
list_tdr

In [ ]:
#その人物IDのショッパーの接触商品の一覧表を作成
df_tmp = df_et3[df_et3["customer_id"].isin(list_tdr)]
df_tmp.head()

In [ ]:
df_tmp["product_name"].value_counts()

## 6.6 仮説5「時間帯によって手に取られる商品が異なる」の検証

### 6.6.2 データで検証

In [ ]:
df_et3.info() # 型の確認

In [ ]:
df_tmp = df_et3.copy() # いったんコピー
df_tmp["in_time"] = pd.to_datetime(df_tmp["in_time"]) # 型変換
df_tmp.info() # 型の確認

In [ ]:
df_tmp["hour"] = df_tmp["in_time"].dt.strftime("%H") # 時のみ切り出してhourに格納
df_tmp.head(5)

In [ ]:
df_tmp.groupby("hour")["product_name"].describe()